## The Zoning Problem 
The aim of this notebook is to generate the optimal zoning by formulating a linear assignment problem and using google
OR-Toolsfor solving the optimization problem.
The agent behaviours are taken from the previous study. The occupation and env lattices are random for the test problem .

### The Test Problem 
There are five agents :[Blue,Green,Yellow,Red,Violet]
The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1]
Total number of voxels in a lattice = x 
Value lattice for the agents are =[Bv],[Gv],[Yv],[Rv],[Vv]

####  Aim 
: To find the best combination of Zones to achieve the maximum occupancy value
#### Steps

1. Generate the lattices , Agent behaviours, agents, 
2. Find the possible origin locations for the agents
3. Simulate occupancy behaviour and retrieve the cost of occupancy for each agent at each position
4. Generate the Cost matrix
5. Use the MIP Solver to optimise the combination to get the permutation matrix 


## Initilization

In [1]:
import os
import itertools
import sys
from math import factorial as fac
sys.path.append("D:/TU_Delft/Msc_Building_Technology/Semester_3/Graduation/Aditya_Graduation_Project_BT/06_Libraries")
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import networkx as nx
np.random.seed(0)
np.set_printoptions(threshold=sys.maxsize)
import networkx as nx
from itertools import combinations
import pickle

## Base Lattices

In [2]:
# loading the lattice from csv
lattice_path = os.path.relpath('Base_lattice_2.csv')
avail_lattice_base = tg.lattice_from_csv(lattice_path)
avail_lattice = avail_lattice_base*1
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice*1), avail_lattice)

In [3]:
flattened_lattice = avail_lattice_base.flatten()

## Seperate Lattice into Buildings 

In [4]:
s = tg.create_stencil("von_neumann", 1, 1)

In [5]:
env_lat_flat = avail_lattice_base.flatten()

adj_matrix = np.zeros((env_lat_flat.size,env_lat_flat.size), dtype=int)

all_vox_neighs_inds = avail_lattice_base.find_neighbours(s)
all_vox_neighs = all_vox_neighs_inds[:, 1:]
all_vox_inds = np.indices(all_vox_neighs.shape)[0]

adj_matrix[all_vox_inds.flatten(), all_vox_neighs.flatten()] = 1

adj_matrix[env_lat_flat==0, :] *= 0
adj_matrix[:, env_lat_flat==0] *= 0

g = nx.from_numpy_array(adj_matrix)

In [6]:
g.remove_nodes_from(list(nx.isolates(g)))

In [7]:
g_cc = nx.algorithms.connected_components(g)

buildingid_lat_flat = env_lat_flat * 0 - 1

for i, nodes in enumerate(g_cc):
    buildingid_lat_flat[list(nodes)] = i

In [8]:
buildingid_lat = tg.to_lattice(buildingid_lat_flat.reshape(avail_lattice_base.shape), avail_lattice_base)
flat_id=buildingid_lat.flatten()

In [9]:
avail_id= np.argwhere(flattened_lattice==1)

In [10]:
avail_id

array([[ 243],
       [ 244],
       [ 245],
       [ 246],
       [ 247],
       [ 248],
       [ 249],
       [ 250],
       [ 251],
       [ 254],
       [ 255],
       [ 256],
       [ 257],
       [ 258],
       [ 259],
       [ 260],
       [ 261],
       [ 262],
       [ 265],
       [ 266],
       [ 267],
       [ 268],
       [ 269],
       [ 270],
       [ 271],
       [ 272],
       [ 273],
       [ 276],
       [ 277],
       [ 278],
       [ 279],
       [ 280],
       [ 281],
       [ 282],
       [ 283],
       [ 284],
       [ 287],
       [ 288],
       [ 289],
       [ 290],
       [ 291],
       [ 292],
       [ 293],
       [ 294],
       [ 295],
       [ 298],
       [ 299],
       [ 300],
       [ 301],
       [ 302],
       [ 303],
       [ 304],
       [ 305],
       [ 306],
       [ 309],
       [ 310],
       [ 311],
       [ 312],
       [ 313],
       [ 314],
       [ 315],
       [ 316],
       [ 317],
       [ 320],
       [ 321],
       [ 322],
       [ 3

## Retrieving ID's of cells in the 4 buildings 

In [11]:
Building_4= np.argwhere(flat_id==2)
Building_3= np.argwhere(flat_id==1)
Building_2= np.argwhere(flat_id==3)
Building_1= np.argwhere(flat_id==0)

## Env Lattices

In [12]:
Bv = np.random.randint(1, 20, (np.shape(avail_lattice)), dtype='l')
Gv = np.random.randint(1, 20, (np.shape(avail_lattice)), dtype='l')
Yv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Rv = np.random.randint(1, 9, (np.shape(avail_lattice)), dtype='l')
Vv = np.random.randint(1, 99, (np.shape(avail_lattice)), dtype='l') 

## Stencils

In [13]:
# creating neighborhood definition
stencil_von_neumann = tg.create_stencil("von_neumann", 1, 1)
stencil_von_neumann.set_index([0,0,0], 0)
#print(stencil_von_neumann)

In [14]:
# creating neighborhood definition 
stencil_squareness = tg.create_stencil("moore", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness[0,:,:] = 0 
stencil_squareness[2,:,:] = 0
stencil_squareness.set_index([0,0,0], 0)
stencil_squareness_t = np.transpose(stencil_squareness) 
#print(stencil_squareness_t)

In [15]:
# creating neighborhood definition 
stencil_squareness_von = tg.create_stencil("von_neumann", 1, 1)
# Reshaping the moore neighbourhood
stencil_squareness_von[0,:,:] = 0 
stencil_squareness_von[2,:,:] = 0
stencil_squareness_von.set_index([0,0,0], 0)
stencil_squareness_von_t = np.transpose(stencil_squareness_von) 
#print(stencil_squareness_von)

In [16]:
stencil_cuboid = tg.create_stencil("moore", 1, 1)
stencil_cuboid.set_index([0,0,0], 0)
#print(stencil_cuboid)

In [17]:
Stencil_Find_neighbours_cuboid = tg.create_stencil("moore", 1, 1)

In [18]:
stencil_full_floor = tg.create_stencil("moore",int(10/2) )*0
stencil_full_floor[:,:,int(10/2)] = 1
stencil_full_floor.set_index([0,0,0], 0)
#print(stencil_full_floor)

## Deriving all possible agent Center points

In [19]:
available_index_list= list(np.argwhere(flattened_lattice==1).flatten())

In [20]:
len(np.argwhere(flattened_lattice==1))

684

In [21]:
len(available_index_list)

684

In [22]:
def all_possible_origins(flat_lattice,number_of_points_needed):
    reshaped_lattice= flat_lattice.reshape(99,4,16)*1
    reshaped_lattice_aggregated_value =np.sum(reshaped_lattice,axis=2)
    array = reshaped_lattice_aggregated_value.flatten()
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    
    indices =list(range(6,flat_lattice.size,16))
    points_1D=[]
    for points in range(number_of_points_needed):
        index= np.where(ranks==points)
        points_1D.append(indices[int(index[0])] )  
    return points_1D


In [23]:
## The number of voxels which each agent has to occupy There are five agents :[Blue,Green,Yellow,Red,Violet]
## The number of voxels which each agent occupies are [B1,G1,Y1,R1,V1] 
mass_size = np.count_nonzero((avail_lattice==1)) 
Bn = 136 # Blue
Gn= 136 # Green
Yn=90 # Yellow
Rn= 90 # Red
Vn= 230 # Violet

In [24]:
#Divide the available lattice into grids to generate the necessary locations for origins??

In [25]:
def ranking_origins(point_id,value_lattice,origin_number):
    selected_neighbours= avail_lattice_base.find_neighbours(Stencil_Find_neighbours_cuboid)[point_id]
    neighbour_vals_for_agent= value_lattice.flatten()[selected_neighbours]
    neighbour_vals_aggregated_for_agent =np.sum(neighbour_vals_for_agent,axis=1)
    all_points = np.array(point_id)
    array = neighbour_vals_aggregated_for_agent
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    
    a_lattice = ranks.flatten()
    Co_ordinates = []
    for item in range(0,origin_number):
        Co_ordinates.append(all_points[np.argwhere(a_lattice==item)].flatten()[0])    
        
    return Co_ordinates

In [26]:
Blue_A_origin_1 = ranking_origins(avail_id.flatten(),Bv,Bn)
Green_A_origin_1 = ranking_origins(avail_id.flatten(),Gv,Gn)
Yellow_A_origin_1 = ranking_origins(avail_id.flatten(),Yv,Yn)
Red_A_origin_1 = ranking_origins(avail_id.flatten(),Rv,Rn)
Violet_A_origin_1 = ranking_origins(avail_id.flatten(),Vv,Vn)

In [27]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of random/argmax occupancy on a 2d squarish stencil 
    def random_occupy_squareness(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
     
      # definition of random/argmax occupancy on a 3d cubish stencil
    def random_occupy_cubish(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
      
        
    def random_occupy_cubish_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_cuboid, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
        
    def argmax_occupy_von_neumann(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_lattice = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_lattice = env.availibility.flatten()[neighs_full_lattice]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_lattice = neighs_full_lattice[neighs_availibility_full_lattice==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_lattice
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)

        
     # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_moore(self, env):
        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        #print(neighs)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_floor, loc = self.origin)

        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]               
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        #print(neighs_availibility)
        
        # find env values of all neighbours
        all_neighs_value = env.value.flatten()[neighs]
        all_neighs_value_mod = np.copy(all_neighs_value)
        
        
        #finding number of neighbours and bumping the values based on adjacency for a 9 neighbourhood
        
        #print(neighbourhood_details)
        one = neighs_availibility[1] + neighs_availibility[2] 
        two = neighs_availibility[0] + neighs_availibility[2] 
        three = neighs_availibility[1] + neighs_availibility[4] 
        four = neighs_availibility[0] + neighs_availibility[6] 
        five = neighs_availibility[2] + neighs_availibility[7] 
        six = neighs_availibility[3] + neighs_availibility[6] 
        seven = neighs_availibility[5] + neighs_availibility[7] 
        eight = neighs_availibility[6] + neighs_availibility[4] 
        neighbourhood_details = [one,two,three,four,five,six,seven,eight]
        
        #print(neighbourhood_details)
        for detail in range(len(neighs_availibility)-1):
            neighbourhood_condition = neighbourhood_details[detail] 
            #print(neighbourhood_condition)
            if neighbourhood_condition == 3:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + one_neighbour_factor
            elif neighbourhood_condition == 4:
                all_neighs_value_mod[detail]= all_neighs_value_mod[detail] + two_neighbour_factor
            else:
                all_neighs_value_mod[detail] = all_neighs_value_mod[detail]
        #print(all_neighs_value_mod)   
        

        neighs_value_flattened = env.value.flatten()
        for val_mod in all_neighs_value_mod:
            for neigh in neighs :
                neighs_value_flattened[neigh]=val_mod
        
        
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = neighs_value_flattened[free_neighs]
        
        #print(free_neighs_value)
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    
    
         # definition of 2d occupying method for agents
    def one_neighbour_occupy_squareness_von_neumann(self, env):
        
         # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        neighs_full_floor = env.availibility.find_neighbours_masked(stencil_full_lattice, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        neighs_availibility_full_floor = env.availibility.flatten()[neighs_full_floor]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        free_neighs_full_floor = neighs_full_floor[neighs_availibility_full_floor==1]
        #print(free_neighs)
        if len(free_neighs)== 0 :
            free_neighs = free_neighs_full_floor
        else: 
            free_neighs= free_neighs
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
       # selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        selected_neigh = np.random.choice(free_neighs,1)
        #print(selected_neigh)
        # update information
        ####################
        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()
        #print(self.origin)
    def one_neighbour_occupy_squareness_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = np.argwhere(neighs_availibility_full_lattice==1).flatten()
 
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_squareness.append(neighs)

        neighs_flatten = np.array(env.neigh_squareness).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        #neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)


        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )

        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
   
        
        
    def one_neighbour_occupy_cubish_behaviour (self,env):        
        value_lattice_flat = env.value.flatten()
        sqr_factor = 10.1
        sqr_shift = 10.0
        init_loc = self.origin
        neighs_availibility_full_lattice = env.availibility.flatten()
        free_neighs_full_lattice = np.argwhere(neighs_availibility_full_lattice==1).flatten()
        
        agn_locs = [list(init_loc)]
        all_neighs =[] 
        avail_lattice_flat = env.availibility.flatten()
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        all_neighs.append(neighs)
        env.neigh_cubish.append(neighs)

        neighs_flatten = np.array(env.neigh_cubish).flatten()
        #print(neighs_flatten)
        neighs_availability = avail_lattice_flat[neighs_flatten]
  
        # keep the available ones only
        avail_neighs = neighs_flatten[neighs_availability==1] 
        
        if len(avail_neighs)== 0 :
            avail_neighs = free_neighs_full_lattice
        else: 
            avail_neighs= avail_neighs
                
        #print(avail_neighs)
        avail_unq_neighs, avail_unq_neighs_count = np.unique(avail_neighs, return_counts=True)
        #print(avail_unq_neighs)
        #print(avail_unq_neighs_count)
        neighs_unq_base_value = value_lattice_flat[avail_unq_neighs]
        
        #neigh_sqr_evaluation = np.power(sqr_factor, (avail_unq_neighs_count - 1)) * neighs_unq_base_value + sqr_shift
        neigh_sqr_evaluation = neighs_unq_base_value + sqr_shift * (avail_unq_neighs_count - 1)

       # print(neighs_unq_base_value) 
        selected_neigh_index = np.argmax(neigh_sqr_evaluation)
        selected_neigh_1D_id = avail_unq_neighs[selected_neigh_index]
        #selected_neigh_3D_id = np.unravel_index(selected_neigh_1D_id,bounds.shape )       
            
        # update information
        ####################
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh_1D_id, env.availibility.shape)).flatten()
        #print(selected_neigh_1D_id)
        #print(free_neighs_full_lattice)

            

In [28]:
# Agent init class

def initialize_agents_random_origin (stencil,avail_lattice):
    #finding origin 
    agn_num = 1
    occ_lattice = avail_lattice*0 -1
    avail_flat = avail_lattice.flatten()
    avail_index = np.array(np.where(avail_lattice == 1)).T
    select_id = np.random.choice(len(avail_index), agn_num)
    agn_origins = tuple(avail_index[select_id].flatten()) 

    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

def initialize_agents_fixed_origin (stencil,avail_lattice,origin):
    #finding origin 
    agn_origins = np.unravel_index(origin,avail_lattice.shape)
    select_id = origin
    # Defining agents
    myagent = agent(agn_origins, stencil, select_id)

    return myagent

In [29]:
# environment class
class environment():
    def __init__(self, lattices, agents,number_of_iterations,method_name):
        self.availibility = lattices["availibility"]
        self.value = lattices["enviornment"]
        self.agent_origin = self.availibility 
        self.agents = agents
        self.update_agents()
        self.number_of_iterations = number_of_iterations
        self.method_name = method_name
        self.neigh_cubish = []
        self.neigh_squareness = []
    def update_agents(self):
        # making previous position available
      #  self.availibility[tuple(self.agents.old_origin)] = self.availibility[tuple(self.agents.old_origin)] * 0 + 1
        # removing agent from previous position
        self.agent_origin[tuple(self.agents.old_origin)] *= 0+1
        # making the current position unavailable
        self.availibility[tuple(self.agents.origin)] = self.agents.id
        # adding agent to the new position 
        self.agent_origin[tuple(self.agents.origin)] = self.agents.id
    
    def random_occupy_squareness_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_squareness(self)
        # update the agent states in environment
        self.update_agents()
        
    def random_occupy_cubish_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish(self)
        # update the agent states in environment
        self.update_agents()
    
    def random_occupy_cubish_von_neumann_agents(self):
        # iterate over egents and perform the walk
        self.agents.random_occupy_cubish_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
    
    def argmax_occupy_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.argmax_occupy_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_moore(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_moore(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_von_neumann(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_von_neumann(self)
        # update the agent states in environment
        self.update_agents()
      
    def one_neighbour_occupy_cubish_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_cubish_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        
    def one_neighbour_occupy_squareness_behaviour(self):
        # iterate over egents and perform the walk
        self.agents.one_neighbour_occupy_squareness_behaviour(self)
        # update the agent states in environment
        self.update_agents()
        

In [30]:
def calculate_cost_for_positions(Location_list,Enviornment_lattice,number_of_cells_to_be_occupied):
    All_viz_options = []
    for indexing, item in enumerate(Location_list):
        #print(indexing)
        Agent = agent(np.unravel_index(item,init_avail_lattice.shape),stencil_cuboid,9)

        occ_lattice_sim = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

        env = {"availibility": occ_lattice_sim,"enviornment": Enviornment_lattice}


        env_1 = environment(env,Agent,number_of_cells_to_be_occupied,"one_neighbour_occupy_cubish_behaviour")

        env_availability_viz = []
        env_list =[env_1]
        number_steps = max(map(lambda e:e.number_of_iterations,env_list))    
        for a in range(number_steps):
            # print(env.availibility)
            #print(env.agent_origin)

            for e in env_list:
                if a < e.number_of_iterations :
                    #print(a)
                    #print(e.number_of_iterations)
                    if e.method_name == "one_neighbour_occupy_squareness_moore":
                        e.one_neighbour_occupy_squareness_moore()

                    elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                        e.one_neighbour_occupy_cubish_agents()

                    elif e.method_name == "random_occupy_squareness_agents" :
                        e.random_occupy_squareness_agents()

                    elif e.method_name == "random_occupy_cubish_agents" :
                        e.random_occupy_cubish_agents()  

                    elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                        e.random_occupy_cubish_von_neumann_agents()                           

                    elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                        e.one_neighbour_occupy_squareness_von_neumann()                


                    elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                        e.one_neighbour_occupy_squareness_behaviour()  

                    elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                        e.one_neighbour_occupy_cubish_behaviour()  

                    elif e.method_name == "argmax_occupy_von_neumann" :
                        e.argmax_occupy_von_neumann()  

        env_availability_viz.append(e.availibility)

        All_viz_options.append(env_availability_viz)

    all_viz_array= np.array(All_viz_options)
    all_viz_array_shape = all_viz_array.shape
    reshaped_all_viz= all_viz_array.reshape(all_viz_array_shape[0]*all_viz_array_shape[1],all_viz_array_shape[2],all_viz_array_shape[3],all_viz_array_shape[4])
    sum_of_values = []
    for lattice in reshaped_all_viz:
        indexes= np.argwhere(lattice.flatten()==9).flatten()
        values= env["enviornment"].flatten()[indexes]
        total = int(np.sum(values))
        sum_of_values.append(total)
    return sum_of_values

In [31]:
"""
Cost_for_PH = np.array(calculate_cost_for_positions(Selected_positions,Private_sector_Owned,20)) # Private Housing
Cost_for_SH = np.array(calculate_cost_for_positions(Selected_positions,Social_Sector_Rental,20)) # Social Housing
Cost_for_FH = np.array(calculate_cost_for_positions(Selected_positions,Free_Sector_rental,20)) # Free sector Housing
Cost_for_RC = np.array(calculate_cost_for_positions(Selected_positions,Restaurants_and_Cafes,20)) # Restaurant and Cafe 
Cost_for_R = np.array(calculate_cost_for_positions(Selected_positions,Offices,20)) # Retail stores
Cost_for_O = np.array(calculate_cost_for_positions(Selected_positions,Retail,20)) # Offices
Cost_for_P = np.array(calculate_cost_for_positions(Selected_positions,Parking,20)) # Parking
"""

'\nCost_for_PH = np.array(calculate_cost_for_positions(Selected_positions,Private_sector_Owned,20)) # Private Housing\nCost_for_SH = np.array(calculate_cost_for_positions(Selected_positions,Social_Sector_Rental,20)) # Social Housing\nCost_for_FH = np.array(calculate_cost_for_positions(Selected_positions,Free_Sector_rental,20)) # Free sector Housing\nCost_for_RC = np.array(calculate_cost_for_positions(Selected_positions,Restaurants_and_Cafes,20)) # Restaurant and Cafe \nCost_for_R = np.array(calculate_cost_for_positions(Selected_positions,Offices,20)) # Retail stores\nCost_for_O = np.array(calculate_cost_for_positions(Selected_positions,Retail,20)) # Offices\nCost_for_P = np.array(calculate_cost_for_positions(Selected_positions,Parking,20)) # Parking\n'

In [32]:
def agent_based_on_origin(Origin,index,stencil):
    return agent(np.unravel_index(Origin,init_avail_lattice.shape),stencil,index)

In [33]:
def abm_simulation(Agent_origin,a_id,stencil,val_lattice,number_of_voxels):
    Agent_sim = agent_based_on_origin(Agent_origin,a_id,stencil)
    env_lat = {"availibility": avail_lattice,"enviornment": val_lattice}
    env_sim = environment(env_lat,Agent_sim,number_of_voxels,"one_neighbour_occupy_cubish_behaviour")
    env_availability_viz = []
    env_list = [env_sim]
    number_steps = max(map(lambda e:e.number_of_iterations,env_list))
    for a in range(number_steps):
        # print(env.availibility)
        #print(env.agent_origin)
        #print(a)
        for e in env_list:
            if a < e.number_of_iterations :
                #print(a)
                #print(e.number_of_iterations)
                if e.method_name == "one_neighbour_occupy_squareness_moore":
                    e.one_neighbour_occupy_squareness_moore()

                elif e.method_name == "one_neighbour_occupy_cubish_agents" :
                    e.one_neighbour_occupy_cubish_agents()

                elif e.method_name == "random_occupy_squareness_agents" :
                    e.random_occupy_squareness_agents()

                elif e.method_name == "random_occupy_cubish_agents" :
                    e.random_occupy_cubish_agents()  

                elif e.method_name == "random_occupy_cubish_von_neumann_agents" :
                    e.random_occupy_cubish_von_neumann_agents()                           

                elif e.method_name == "one_neighbour_occupy_squareness_von_neumann" :
                    e.one_neighbour_occupy_squareness_von_neumann()                


                elif e.method_name == "one_neighbour_occupy_squareness_behaviour" :
                    e.one_neighbour_occupy_squareness_behaviour()  

                elif e.method_name == "one_neighbour_occupy_cubish_behaviour" :
                    e.one_neighbour_occupy_cubish_behaviour()  

                elif e.method_name == "argmax_occupy_von_neumann" :
                    e.argmax_occupy_von_neumann()  

                env_availability_viz.append(e.availibility-1)
    return env_availability_viz

In [34]:
mask_Violet = avail_lattice.flatten()[Violet_A_origin_1]==1
available_origin_Violet= np.array(Violet_A_origin_1)[mask_Violet]

In [35]:
Violet_simulation= abm_simulation(available_origin_Violet[0],500,stencil_cuboid,Vv,Vn+1)

In [36]:
p = pv.Plotter(notebook=True)

base_lattice = Violet_simulation[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = Violet_simulation[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 300
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[3 3 3]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(89.32480766111564, 111.82480766111564, 96.82480766111564),
 (7.5, 30.0, 15.0),
 (0.0, 0.0, 1.0)]

In [37]:
mask_Blue = avail_lattice.flatten()[Blue_A_origin_1]==1
available_origin_FS= np.array(Blue_A_origin_1)[mask_Blue]

In [38]:
Blue_simulation= abm_simulation(Blue_A_origin_1[0],100,stencil_cuboid,Bv,Bn-1)

In [39]:
p = pv.Plotter(notebook=True)

base_lattice = Blue_simulation[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = Blue_simulation[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 1500
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[3 3 3]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(89.32480766111564, 111.82480766111564, 96.82480766111564),
 (7.5, 30.0, 15.0),
 (0.0, 0.0, 1.0)]

In [40]:
mask_Green = avail_lattice.flatten()[Green_A_origin_1]==1
available_origin_Green= np.array(Green_A_origin_1)[mask_Green]

In [41]:
available_origin_Green

array([ 289, 1042,  859, 1054, 1050, 1043, 1065,  980, 1123, 1020, 1031,
        755,  291, 1032,  269,  248, 1080,  280, 1064,  278,  779,  744,
        270,  281,  290,  500,  279, 1041,  819, 1138,  756,  268,  267,
       1129,  969,  789,  812, 1051, 1010,  981,  768,  247,  393,  959,
        749,  834,  581, 1060,  522,  754, 1139,  757,  767,  245, 1137,
        958,  987,  249,  790,  592,  984,  823], dtype=int64)

In [42]:
Green_simulation= abm_simulation(available_origin_Green[0],200,stencil_cuboid,Gv,Gn-1)

In [43]:
p = pv.Plotter(notebook=True)

base_lattice = Green_simulation[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = Green_simulation[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 140
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[3 3 3]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(89.32480766111564, 111.82480766111564, 96.82480766111564),
 (7.5, 30.0, 15.0),
 (0.0, 0.0, 1.0)]

In [44]:
mask_Yellow = avail_lattice.flatten()[Yellow_A_origin_1]==1
available_origin_Yellow= np.array(Yellow_A_origin_1)[mask_Yellow]

In [45]:
Yellow_simulation= abm_simulation(available_origin_Yellow[0],300,stencil_cuboid,Yv,Yn-1)

In [46]:
p = pv.Plotter(notebook=True)

base_lattice = Yellow_simulation[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = Yellow_simulation[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 140
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[3 3 3]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(89.32480766111564, 111.82480766111564, 96.82480766111564),
 (7.5, 30.0, 15.0),
 (0.0, 0.0, 1.0)]

In [47]:
mask_Red = avail_lattice.flatten()[Red_A_origin_1]==1
available_origin_Red= np.array(Red_A_origin_1)[mask_Red]

In [48]:
Red_simulation= abm_simulation(available_origin_Red[0],400,stencil_cuboid,Rv,Rn-1)

In [49]:
p = pv.Plotter(notebook=True)

base_lattice = Red_simulation[0]
print(base_lattice.unit)
# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the avilability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa") 

def create_mesh(value):
    f = int(value)
    lattice = Red_simulation[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([1.1, avail_lattice.size])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return
number_steps_2 = 140
p.add_slider_widget(create_mesh, [0, number_steps_2], title='Time', value=0, event_type="always", style="classic")


p.show(use_ipyvtk=True)

[3 3 3]


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(89.32480766111564, 111.82480766111564, 96.82480766111564),
 (7.5, 30.0, 15.0),
 (0.0, 0.0, 1.0)]

In [50]:
"""
for i, lattice in enumerate(Restaurants_simulation):
    csv_path = os.path.relpath('csv/8_abm_f_'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)
"""

"\nfor i, lattice in enumerate(Restaurants_simulation):\n    csv_path = os.path.relpath('csv/8_abm_f_'+ f'{i:03}' + '.csv')\n    lattice.to_csv(csv_path)\n"

In [51]:
len(np.argwhere(avail_lattice.flatten()==1))

1

In [52]:
BC_index = np.argwhere(avail_lattice.flatten()==100).flatten()
GC_index = np.argwhere(avail_lattice.flatten()==200).flatten()
YC_index = np.argwhere(avail_lattice.flatten()==300).flatten()
RC_index = np.argwhere(avail_lattice.flatten()==400).flatten()
VC_index = np.argwhere(avail_lattice.flatten()==500).flatten()

In [53]:
BC= np.sum(Bv.flatten()[BC_index])
GC= np.sum(Gv.flatten()[GC_index])
YC= np.sum(Yv.flatten()[YC_index])
RC= np.sum(Rv.flatten()[RC_index])
VC= np.sum(Vv.flatten()[VC_index])
total_cost = BC + GC + YC+ RC +  VC

In [54]:
YC

480

In [55]:
total_cost

16548

In [56]:

#Zoning_mtrx = pickle.dump( avail_lattice, open( "Zoning output.p", "wb" ) )
#pickle_matrix = pickle.load( open( "dist_matrix_toy_problem.p", "rb" ) ) 

In [57]:
#type(avail_lattice)

In [58]:
#avail_lattice.centroids 

In [64]:
for i, lattice in enumerate(Red_simulation):
    csv_path = os.path.relpath('csv_sequential/abm_f_E'+ f'{i:03}' + '.csv')
    lattice.to_csv(csv_path)